In [5]:
# Función de costo y exatitud.
import collections
from sklearn import metrics
from sklearn.metrics import roc_auc_score


def custom_cost(modelo, X, y, printout=True):
    predicciones = modelo.predict(X)
    return _custom_cost(y, predicciones, printout)
    
def _custom_cost(y, predicciones, printout=True):

    accuracy = metrics.accuracy_score(y, predicciones)
    precision = metrics.precision_score(y, predicciones)
    recall = metrics.recall_score(y, predicciones)

    matriz_confusion = metrics.confusion_matrix(y, predicciones)
    # Mapping from http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    # 0 is the negative category.
    # rows are the actual values.
    # columns are the predicted values.
    # So it is just the opposite orientation of +/-.
    TN = matriz_confusion[0,0]
    FN = matriz_confusion[1,0]
    FP = matriz_confusion[0,1]
    TP = matriz_confusion[1,1]
    
    AUC = roc_auc_score(y, predicciones)

    # From this we can calculate the other measures.
    # If cost for each state, we can calculate overall cost.
    # Coefficients de costo.
    cTP = -1; cFN = 5; cFP = 1; cTN = 0
    todo_costo = cFN*FN + cFP*FP + cTN*TN + cTP*TP
    cost = todo_costo/len(y)
    
    if printout:
        print('Exactitud:    ',  accuracy)
        print('Precisión:    ', precision)
        print('Exhaustividad:', recall)
        print('AUC:          ', AUC)
        print('Costo por cliente:', cost)
            
    quality = collections.namedtuple('Quality', 'accuracy, precision, recall, AUC, cost')
    return quality(accuracy, precision, recall, AUC, cost)


'\n# define the function to estimate empirical probabilities.\n# returns predicted probability (trimmed by n on both sides)\n# and empirical probability calculated by Yeh\'s SMM method.\nimport numpy as np\ndef getPQprobs(y, y_prob, n=50):\n    frame = { \'y\': y, \'y_prob\': y_prob } \n    df = pd.DataFrame(frame)\n    df = df.sort_values([\'y_prob\'])\n    y = df.y\n    y_prob = df.y_prob\n    prob_smm = []\n    denom = 2*n + 1\n    for i in range(n, len(y)-n):\n        p = y[i-n:i+n+1].sum()/denom\n        prob_smm.append(p)\n    prob_pred = np.array(y_prob[n:len(y)-n])\n    prob_smm = np.array(prob_smm)\n    return (prob_pred, prob_smm)\n\n# plot actual versus predicted probabilities sorted ascending on predicted.\nimport matplotlib.pyplot as plt\n%matplotlib inline\ndef plotPQ(prob_emp, prob_pred):\n    plt.rcParams.update({\'font.size\': 9})\n    plt.plot(prob_pred, prob_emp, label= ("Empirical x Prediccion") )\n    plt.xlabel("Prob Predicción")\n    plt.ylabel("Prob Empirical")\